In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv("C:/Users/mail2/OneDrive/Desktop/Ml_Data/NLP_DATA/medicine.csv")

In [3]:
data.head(1)

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,Acne,Mild to moderate acne (spots)


In [4]:
#data.info()
#data.duplicated().sum()

In [5]:
#tokenization
data["Description"][0]

'Mild to moderate acne (spots)'

In [6]:
data["Description"]=data["Description"].apply(lambda x:x.split(' '))
data["Reason"]=data["Reason"].apply(lambda x:x.split(' '))

In [7]:
data.head(1)

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,[Acne],"[Mild, to, moderate, acne, (spots)]"


In [8]:
data['Description'] = data['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [9]:
#data['Description'] = data['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [10]:
data.head(1)

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,[Acne],"[Mild, to, moderate, acne, (spots)]"


In [11]:
data["Reason_Description"]=data["Description"]+data["Reason"]

In [12]:
data.head(1)

,index,Drug_Name,Reason,Description,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,[Acne],"[Mild, to, moderate, acne, (spots)]","[Mild, to, moderate, acne, (spots), Acne]"


In [13]:
data_recomm=data[["index","Drug_Name","Reason_Description"]]

In [14]:
data_recomm.head(1)

,index,Drug_Name,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,"[Mild, to, moderate, acne, (spots), Acne]"


In [15]:
data_recomm["Reason_Description"]=data_recomm["Reason_Description"].apply(lambda x:" ".join(x))

In [16]:
data_recomm.head(1)

,index,Drug_Name,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,Mild to moderate acne (spots) Acne


In [35]:
data_recomm["Reason_Description"]=data_recomm["Reason_Description"].apply(lambda x:x.lower())

In [43]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [44]:
ps_stemmer=PorterStemmer()
ps_lemmatizer = WordNetLemmatizer()

In [60]:
def porter_step(text):
    list_y=[]
    for i in text.split():
        list_y.append(ps_lemmatizer.lemmatize(i))
    return " ".join(list_y)

In [61]:
data_recomm["Reason_Description"]=data_recomm["Reason_Description"].apply(porter_step)

In [62]:
data_recomm.head(1)

,index,Drug_Name,Reason_Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,mild to moder acn (spots) acn


In [63]:
#tfidf

In [64]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [65]:
tf_id_vector=TfidfVectorizer(use_idf=True,ngram_range=(2,2),stop_words="english")

In [66]:
vector=tf_id_vector.fit_transform(data_recomm["Reason_Description"]).toarray()

In [73]:
#vector.shape

In [74]:
#tf_id_vector.get_feature_names()

In [76]:
#vector_similarity
from sklearn.metrics.pairwise import cosine_similarity

In [78]:
vector_similarity=cosine_similarity(vector)

In [80]:
#vector_similarity[1]

In [82]:
def med_recomm(medicine):
    med_index=data_recomm[data_recomm["Drug_Name"]==medicine].index[0]
    distance=vector_similarity[med_index]
    med_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in med_list:
        print(data_recomm.iloc[i[0]].Drug_Name)

In [85]:
data["Drug_Name"][1]

'A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0.025% Gel 20gm'

In [86]:
med_recomm('A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0.025% Gel 20gm')

Acnedap Plus Gel 15gm
Acnetoin Plus Oinment 15gm
Acno 0.05% Gel 15gmAcno 20mg Capsule 10'S
Adiff Aqueous Gel 15gm
Alesa 2.5% Gel 1gmAlesa 5% Gel 1gm
